# Introducción a `scikit-learn`

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

* Recorrido por las posibilidades de la librería ([guías de usuario](http://scikit-learn.org/stable/user_guide.html))
* Familizarización con la documentación ([API](http://scikit-learn.org/stable/modules/classes.html))
* **Tarea**: Implementación de un flujo de trabajo sencillo para regresión (parecido al [tutorial básico](http://scikit-learn.org/stable/tutorial/basic/tutorial.html))

Estas notas pueden encontrarse en el repositorio https://github.com/matiasbattocchia/clases-aprendizaje-automatico.

Comunidad de ciencia de datos en Argentina: https://www.facebook.com/groups/DataScienceArgentina.

### Utilidad

* Aprendizaje supervisado
    * Clasificación
    * Regresión
* Aprendizaje no supervisado
* ~~Aprendizaje por refuerzos~~

Redes neuronales: solo perceptrón multicapa y *restricted Boltzmann machine*.

### Extensiones

http://scikit-learn.org/stable/related_projects.html

* Pandas
* Más algoritmos
* **Automatización** 😀
* Dominios específicos
    * Visión computarizada (imágenes)
    * Procesamiento del lenguaje (texto)
    * Medicina, astronomía, geografía...


### Datos

`scikit-learn` consume **datos** con forma de matriz o arreglo bidimensional, de dimensión `(n_muestras, n_atributos)` — es como imaginamos normalmente a los datos, dispuestos en una tabla donde las **columnas** son los atributos y hay tantas muestras como **filas**.

Convencionalmente en la documentación la varible `X` se utiliza para los **atributos** propiamente dichos, y la variable `y` para los **objetivos**. Cuando el objetivo es uno solo, `y` suele tomar la forma de arreglo unidimensional de dimensión `(n_muestras,)`. 

### Objetos

En `scikit-learn` hay dos tipos fundamentales de objetos:

* Los **transformadores**, que implementan los métodos
    * `fit(X, y)` y
    * `transform(X)`,

* y los **estimadores**, que implementan
    * `fit(X, y)`,
    * `predict(X)`.

### Aprendizaje supervisado
    
- 1.1. **Generalized Linear Models**
- 1.2. Linear and Quadratic Discriminant Analysis
- 1.3. Kernel ridge regression
- 1.4. **Support Vector Machines**
- 1.5. Stochastic Gradient Descent
- 1.6. **Nearest Neighbors**
- 1.7. Gaussian Processes
- 1.8. Cross decomposition
- 1.9. **Naive Bayes**
- 1.10. **Decision Trees**
- 1.11. Ensemble methods
- 1.12. Multiclass and multilabel algorithms
- 1.13. Feature selection
- 1.14. Semi-Supervised
- 1.15. Isotonic regression
- 1.16. Probability calibration
- 1.17. Neural network models (supervised)

### Aprendizaje no supervisado

- 2.1. Gaussian mixture models
- 2.2. Manifold learning
- 2.3. **Clustering**
- 2.4. Biclustering
- 2.5. **Decomposing signals in components** (matrix factorization problems)
- 2.6. Covariance estimation
- 2.7. Novelty and Outlier Detection
- 2.8. Density Estimation
- 2.9. Neural network models (unsupervised)

# Flujo de trabajo

<img src='https://docs.google.com/drawings/d/1HJH4Al7gkcIKOr21w-ciZwAFZad6CsU_YKdeAiHHolA/pub?w=960&h=720' alt="flujo de trabajo" style="width: 700px;"/>

## Conjunto de datos de plantas de iris

**Cantidad de instancias**: 150
   	
**Atributos** (4)
    1. Largo del sépalo [cm]
    2. Ancho del sépalo [cm]
    3. Largo del pétalo [cm]
    4. Ancho del pétalo [cm]
    
**Objetivos** (1)
    5. Clase
        - Setosa
        - Versicolour
        - Virginica

**Valores ausentes**: No

<img src='https://upload.wikimedia.org/wikipedia/commons/4/41/Iris_versicolor_3.jpg' alt="flor de iris" style="width: 400px;"/>

In [8]:
from sklearn.datasets import load_iris

iris = datasets.load_iris
X, y = iris.data, iris.target

print('X: datos   ', X.shape)
print('y: objetivo', y.shape)

X: datos    (150, 4)
y: objetivo (150,)


## Preprocesamiento de datos

### Carga de datos\*

Integrando `Pandas` con `scikit-learn` usando el paquete [`sklearn-pandas`](https://github.com/pandas-dev/sklearn-pandas).

```
# pip install sklearn-pandas
```

In [16]:
import sklearn.preprocessing
from sklearn_pandas import DataFrameMapper

data = pd.DataFrame({'mascota': ['gato', 'perro', 'perro', 'pez'],
                     'hijos':   [ 4.,  6,  3,  3],
                     'salario': [90., 24, 44, 27]})

data

,hijos,mascota,salario
0,4.0,gato,90.0
1,6.0,perro,24.0
2,3.0,perro,44.0
3,3.0,pez,27.0


In [17]:
mapper = DataFrameMapper([
    ('mascota', sklearn.preprocessing.LabelBinarizer()),
    (['hijos'], sklearn.preprocessing.StandardScaler())
], df_out=True)

mapper.fit_transform(data)

,mascota_gato,mascota_perro,mascota_pez,hijos
0,1.0,0.0,0.0,0.000000
1,0.0,1.0,0.0,1.632993
2,0.0,1.0,0.0,-0.816497
3,0.0,0.0,1.0,-0.816497


### Limpieza de datos

* Cardinalidad
* Rango
* Desviación
* Tipo
    * Booleano
    * Numéro (separadores)
    * Texto
        * espacios (*trimming*)
        * tildes
        * casos (mayúsculas, minúsculas)
* **Codificación** (UTF-8, etcétera)

### Partición del conjunto de datos

* Entrenamiento (50%)
* Validación (25%) — salvo cross-validation o ausencia de hiperparámetros.
* Prueba (25%)

**Ejemplo**: [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [65]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

### Muestreo\* — conjunto desbalanceado

El *entrenamiento* y la *validación* de estimadores suele requerir conjuntos de datos **balanceados**; no así la *prueba* de los mismos ya que deben enfrentar datos reales del problema (**desbalanceados**).

`scikit-learn` apenas provee algoritmos de muestro, podemos usar la extensión [`imbalanced-learn`](http://contrib.scikit-learn.org/imbalanced-learn/index.html) que implementa varios.

    # pip install imbalanced-learn

`imbalanced-learn` aporta objetos del tipo **muestreador** (*sampler*) que implementan los métodos
* `fit(X, y)` y
* `sample(X, y)`.

Algunos algoritmos:
* Under-sampling
  * ClusterCentroids
  * RandomUnderSampler
* Over-sampling
  * SMOTE
  * RandomOverSampler
  
**Nota**: `imbalanced-learn` re-implementa la clase `Pipeline` para que admita muestreadores.

**Ejemplo**: [`RandomUnderSampler`](http://contrib.scikit-learn.org/imbalanced-learn/generated/imblearn.under_sampling.RandomUnderSampler.html)

In [18]:
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler

# generación de un conjunto de datos
X, y = make_classification(n_classes=2, class_sep=2, weights=[0.1, 0.9],
                           n_informative=3, n_redundant=1, flip_y=0,
                           n_features=20, n_clusters_per_class=1,
                           n_samples=200, random_state=10)

# aplicación de random under-sampling
rus = RandomUnderSampler()
X_resampled, y_resampled = rus.fit_sample(X, y)

---

## Preprocesamiento de atributos

- 4.1. Pipeline and FeatureUnion: combining estimators
- 4.2. Feature extraction
- 4.3. Preprocessing data
- 4.4. Unsupervised dimensionality reduction
- 4.5. Random Projection
- 4.6. Kernel Approximation
- 4.7. Pairwise metrics, Affinities and Kernels
- 4.8. Transforming the prediction target (y)

### Extracción

4.2 http://scikit-learn.org/stable/modules/feature_extraction.html

* Imágenes
* Lenguaje

### Transformación

4.3.1 http://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling

* Estandarización (`StandardScaler`) — A cada atributo le remueve su valor medio y lo escala dividiéndolo por su desviación estándar.
  * Centrar los datos es prácticamente obligatorio (hay excepciones).
  * Normalizar los datos solo si los atributos difieren en unidades y/u órdenes de magnitud.
* Reajuste
    * Rango (`MinMaxScaler`)
    * Valor absoluto (`MaxAbsScaler`)

**Ejemplo**: [`StandardScaler`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [66]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler(with_mean=True, with_std=True)
std.fit(X_train)
X_train_std = std.transform(X_train)     

![](http://cs231n.github.io/assets/nn2/prepro1.jpeg)

4.3.2 http://scikit-learn.org/stable/modules/preprocessing.html#normalization

* Normalización (Normalizer) — Divide vectores por su norma (afecta filas en vez de columnas).

### Imputación de valores ausentes

4.3.5 http://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values

* Descarte (tirar la muestra)
* **Valor más común**
* **Valor medio**
* **Valor mediano**
* Estimación (clasificación/regresión)
* *Hot-deck* (el valor de la muestra más parecida)
* Valor ausente (*NA*) como otro valor

### Creación

4.3.6 http://scikit-learn.org/stable/modules/preprocessing.html#generating-polynomial-features

De $(X_1, X_2)$ a $(1, X_1, X_2, X_1^2, X_1X_2, X_2^2)$.

### Reducción de dimensionalidad

4.4 http://scikit-learn.org/stable/modules/unsupervised_reduction.html

* PCA — análisis de componentes principales. Estandarizar los datos antes de usar PCA.
* Casi todos los estimadores **no supervisados** implementan el método `transform(X)`.
* [Algunos](http://scikit-learn.org/stable/modules/lda_qda.html) estimadores **supervisados** también.

**Ejemplo**: [`PCA`](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)

In [57]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2, whiten=False)
pca.fit(X_train_std)
X_train_pca = pca.transform(X_train_std)

![](http://cs231n.github.io/assets/nn2/prepro2.jpeg)

### Selección — solo aprendizaje supervisado

1.13 http://scikit-learn.org/stable/modules/feature_selection.html

* Umbral de varianza
* Análisis univariado
* Usando un estimador
* Eliminación recursiva (también existe la agregación recursiva)

**Ejemplo**: [`SelectKBest`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)

In [55]:
from sklearn.feature_selection import SelectKBest

kbest = SelectKBest(k=1)
kbest.fit(X_train_std, y_train)
X_train_kbest = kbest.transform(X_train_std)

---

## Selección de modelos

- 3.1. Cross-validation: evaluating estimator performance
- 3.2. Tuning the hyper-parameters of an estimator
- 3.3. Model evaluation: quantifying the quality of predictions
- 3.4. Model persistence
- 3.5. Validation curves: plotting scores to evaluate models

![](http://scikit-learn.org/stable/_static/ml_map.png)

¿Qué estimador usar para el conjunto de datos de **plantas de iris**?

1. ¿Más de 50 muestras? Sí, el conjunto de datos tiene 150 (en realidad un poco menos porque hemos separado un conjunto de prueba).
2. ¿Hay que predecir una categoría? Sí, queremos predecir a qué especie pertenece cada planta.
3. ¿Los datos estás anotados? Sí, están anotados en tres categorías.
4. ¿Más de 100,000 muestras? No...

Nos recomiendan usar una máquina de vectores de soporte ([*support vector machine*](https://en.wikipedia.org/wiki/Support_vector_machine)) con un *kernel* lineal.

**Ejemplo**: [`SVC`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

In [105]:
from sklearn.svm import SVC

estimador = SVC(kernel='linear', C=1, probability=True)
estimador.fit(X_train_std, y_train)

X_test_std = std.transform(X_test)

y_pred = estimador.predict(X_test_std)

### `predict_proba(X)`

1.6 http://scikit-learn.org/stable/modules/calibration.html

When performing classification you often want not only to predict the class label, but also obtain a probability of the respective label. This probability gives you some kind of confidence on the prediction. Some models can give you poor estimates of the class probabilities and some even do not support probability prediction. The calibration module allows you to better calibrate the probabilities of a given model, or to add support for probability prediction.

In [121]:
estimador.predict_proba(X_test[:3]).round()

array([[ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.]])

### Cantidad de objetivos — solo aprendizaje supervisado

1.12 http://scikit-learn.org/stable/modules/multiclass.html

* Clasificador
    * Binario
    * Multi
        * Clase
        * Etiqueta
        * Clase-etiqueta
    
* Regresor
    * Univariado
    * Multivariado
    
**En `scikit-learn` todos los clasificadores aceptan varias clases**. Algunos estimadores trabajan inherentemente con múltiples objetivos y le sacan provecho a la correlación entre los mismos. Cuando no es el caso del estimador, existen diferentes estrategias para que admita múltiples objetivos:
* *OVO* (uno-contra-uno),
* *OVA* (uno-contra-todos).

### Ensambles — solo aprendizaje supervisado

1.11 http://scikit-learn.org/stable/modules/ensemble.html

Diferentes tipos de ensambles:
* Promediadores: estimadores en paralelo, reducen la **varianza**.
* Propulsores (*boosting*): estimadores en serie, reducen el **sesgo**.

### Evaluación del modelo

3.3 http://scikit-learn.org/stable/modules/model_evaluation.html

Cada estimador implementa un método llamado `score(X, y)` que devuelve un puntaje del desempeño del estimador. El puntaje es calculado usando una métrica acorde a la naturaleza del estimador, por ejemplo los regresores suelen reportar **R²** mientas que los clasificadores, **efectividad**.

In [70]:
estimador.score(X_test_std, y_test)

0.97368421052631582

[`Efectividad`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)

In [88]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.97368421052631582

[`Matriz de confusión`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)

https://en.wikipedia.org/wiki/Confusion_matrix

In [79]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[13,  0,  0],
       [ 0, 15,  1],
       [ 0,  0,  9]])

[`Reporte`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

In [77]:
from sklearn.metrics import classification_report

clases = ['Setosa', 'Versicolour', 'Virginica']
print(classification_report(y_test, y_pred, target_names=clases))

             precision    recall  f1-score   support

     Setosa       1.00      1.00      1.00        13
Versicolour       1.00      0.94      0.97        16
  Virginica       0.90      1.00      0.95         9

avg / total       0.98      0.97      0.97        38



[`F1`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

$F_1 = 2 \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}{\mathrm{precision} + \mathrm{recall}}$

In [86]:
from sklearn.metrics import f1_score

f1_score(y_test, y_pred, average='weighted')  

0.97395228308462156

[`Kappa de Cohen`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html)

https://en.wikipedia.org/wiki/Cohen's_kappa

$\kappa = \frac{p_o - p_e}{1 - p_e}$

In [87]:
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(y_test, y_pred)

0.95978835978835975

### Flujo de datos

![](https://docs.google.com/drawings/d/1aabA_HusMS9sjpVGhFxgYtsKyTypsLz9qd5BqqwAL-U/pub?w=1922&h=663)

### Pipeline

4.1 http://scikit-learn.org/stable/modules/pipeline.html

Todos los objetos del flujo, excepto el último, deben ser **transformadores** (deben implementar el método `transform`). El último objeto puede ser de cualquier tipo.

In [100]:
from sklearn.pipeline import make_pipeline

flujo = make_pipeline(StandardScaler(), SVC())
flujo.fit(X_train, y_train)
flujo.score(X_test, y_test)

0.97368421052631582

### Validación cruzada

3.1 http://scikit-learn.org/stable/modules/cross_validation.html

Se necesitan dos cosas:
1. Una estrategia de particionamiento de los datos.
2. Una métrica de evaluación.

**Estrategias**
* K-fold, stratified k-fold — estrategias por defecto para regresores y clasificadores respectivamente.
* Leave one out (LOO)
* Leave P out (LPO)
* Shuffle & split, stratified shuffle & split

<img src='https://docs.google.com/drawings/d/1EoJOIjEnVkVGYDyknO7ecsk7zUucGRMrHp8yJgpaYuQ/pub?w=826&h=405' alt="validación cruzada" style="width: 700px;"/>

**Métricas**
* De no especificarse ninguna, se usa el método `score(X, y)` del estimador.
* Las métricas más comunes se pueden pasar como opción, ver [tabla](http://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values).
* Se pueden armar [**puntuadores**](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html) a partir de cualquier métrica, tanto de la API como definidas por el usuario.

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

In [101]:
from sklearn.model_selection import cross_val_score

resultados = cross_val_score(flujo, X_train, y_train, cv=5, scoring='f1_weighted')

print('F1 promedio: %0.2f (+/- %0.2f)' % (resultados.mean(), resultados.std() * 2))

F1 promedio: 0.96 (+/- 0.08)


### Optimización

3.2 http://scikit-learn.org/stable/modules/grid_search.html

In [99]:
from sklearn.model_selection import GridSearchCV

parámetros = {
    'svc__kernel':('linear', 'rbf'),
         'svc__C':[1, 10]
}

grilla = GridSearchCV(flujo, parámetros)
grilla.fit(X_train, y_train)
    
estimador = grilla.best_estimator_

### Evaluación final

Una vez elegido el modelo y ajustados sus hiperparámetros, si se desea los conjuntos de datos de entrenamiento y de validación pueden fusionarse en un nuevo conjunto de entrenamiento para reentrenar el modelo final usando más datos — de hecho es lo que hace `GridSearchCV` para el mejor estimador.

En cambio por más seguridad que se tenga del desempeño del modelo, no es recomendable usar los datos del conjunto de prueba, es mejor usarlos para medir su desempeño y asegurarnos de que esté libre de *bugs*.

In [ ]:
y_pred = estimador.predict(X_test)
f1_score(y_test, y_pred, average='weighted')

### Persistencia del modelo

3.4 http://scikit-learn.org/stable/modules/model_persistence.html

In [102]:
import pickle

with open('modelo.pickle', 'wb') as archivo:
    pickle.dump(estimador, archivo)

with open('modelo.pickle', 'rb') as archivo:
    estimador = pickle.load(archivo)

---

## Boston price data set

**Cantidad de instancias**: 506
   	
**Atributos** (13)
    1.  CRIM per capita crime rate by town
    2.  ZN proportion of residential land zoned for lots over 25,000 sq.ft.
    3.  INDUS proportion of non-retail business acres per town
    4.  CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
    5.  NOX nitric oxides concentration (parts per 10 million)
    6.  RM average number of rooms per dwelling
    7.  AGE proportion of owner-occupied units built prior to 1940
    8.  DIS weighted distances to five Boston employment centres
    9.  RAD index of accessibility to radial highways
    10. TAX full-value property-tax rate per 10,000 USD
    11. PTRATIO pupil-teacher ratio by town
    12. B 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
    13. LSTAT % lower status of the population

**Objetivos** (1)
    14. MEDV Median value of owner-occupied homes in 1000’s USD

**Valores ausentes**: No

In [3]:
from sklearn.datasets import load_boston

boston = load_boston()
X, y   = boston.data, boston.target

print('X: datos   ', X.shape)
print('y: objetivo', y.shape)

X: datos    (506, 13)
y: objetivo (506,)


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

**Sugerencias**:
* 1.1 [Modelo lineal](http://scikit-learn.org/stable/modules/linear_model.html)
  * 1.1.1. Ordinary least squares
  * 1.1.2. Ridge regression
  * 1.1.3. Lasso

In [23]:
# implementar estimador

In [22]:
from sklearn.metrics import r2_score

r2_score(y_test, y_pred) # mientras más cerca de 1.0, mejor

0.89914051165600339

---

## Automatización

[TPOT](https://github.com/rhiever/tpot) es una herramienta de aprendizaje automático automatizado que optimiza el flujo de trabajo.

    # pip install tpot

In [12]:
from tpot import TPOTRegressor

tpot = TPOTRegressor(generations=3, population_size=20)
tpot.fit(X_train, y_train)

y_pred = tpot.predict(X_test)

In [13]:
from sklearn.metrics import r2_score

r2_score(y_test, y_pred) # mientras más cerca de 1.0, mejor

0.89914051165600339

In [9]:
tpot.export('tpot_boston_pipeline.py')